In [1]:
import pandas as pd
import numpy as np
import uuid
from collections import Counter
import math
from progressbar import ProgressBar, AdaptiveETA, Percentage, Bar, RotatingMarker

In [2]:
df = pd.read_hdf('../data/processed/epi_vector.h5')

In [3]:
dat = df.values[:,:-8]

In [4]:
columns = list(df)
print(dat[0])
new_vec = []
for recipe in dat:
    ind = uuid.uuid4().hex
    for i in range(len(recipe)):
        if recipe[i] != 0:
            new_vec.append([ind, columns[i]])

[0.0 0.0 0.0 ... 0.0 0.0 0.0]


In [5]:
basketized = pd.DataFrame(new_vec, columns=["index", "name"]).set_index('index', drop=False)
del basketized["index"]
basketized.head()

,name
index,
2038b17f6fb44f05932fd8a923b12f1f,coarsely grated sharp Cheddar
2038b17f6fb44f05932fd8a923b12f1f,garlic clove
b1389fafb4034eb9a6ca9df00a2e56e8,almond
b1389fafb4034eb9a6ca9df00a2e56e8,apricot jam
b1389fafb4034eb9a6ca9df00a2e56e8,brandy


In [6]:
def generate_candidate_set(size, L):
    candidate_set = set()
    for i in range(len(L)):
        for j in range(i+1, len(L)):
            diff = L[i].difference(L[j])
            if len(diff) == 1:
                candidate_set.add(frozenset(L[i].union(L[j])))
    return candidate_set

def apriori(frame, support_per):
    candidate_sets = []
    frequent_sets = []
    thresh = math.floor(len(frame.groupby(level=0))*support_per)
    
    C1 = Counter()
    for ind, basket in frame.groupby(level=0):
        for item in basket.values:
            C1[item[0]] += 1
    
    L1 = []
    for item in C1.most_common():
        if item[1] > thresh:
            L1.append(set([item[0]]))
    print(L1)
    
    count = Counter()
    candidate = generate_candidate_set(2, L1)
    frequent = []
    size = 2
    while len(candidate) != 0:
        widgets = ['Counting C{}: '.format(size), Percentage(), ' ', Bar(marker=RotatingMarker()), ' ', AdaptiveETA()]
        pbar = ProgressBar(widgets=widgets)
        for ind, basket in pbar(frame.groupby(level=0)):
            for c in candidate:
                if basket["name"].isin(c).sum() == len(c):
                    count[c] += 1
        for item in count.most_common():
            if item[1] > thresh:
                frequent.append(set([item[0]]))
        candidate_sets.append(candidate)
        frequent_sets.append(frequent)
        size += 1
        count = Counter()
        candidate = generate_candidate_set(size, frequent)
        frequent = []
        
    return candidate_sets, frequent_sets

In [7]:
c_sets, f_sets = apriori(basketized, 0.05)

[{'salt'}, {'olive oil'}, {'sugar'}, {'egg'}, {'garlic'}, {'unsalted butter'}, {'onion'}, {'water'}, {'lemon juice'}, {'black pepper'}, {'all - purpose flour'}, {'vegetable oil'}, {'unsalte butter'}, {'milk'}, {'butter'}, {'vanilla extract'}, {'ginger'}, {'shallot'}, {'chicken broth'}, {'garlic clove'}, {'cinnamon'}, {'thyme'}, {'carrot'}, {'baking powder'}, {'tomato'}, {'lime juice'}, {'all purpose flour'}, {'parsley'}]


Counting C2: 100% ||||||||||||||||||||||||||||||||||||||||||||||| Time: 0:43:01
Counting C3: 100% ||||||||||||||||||||||||||||||||||||||||||||||| Time: 0:13:54


In [8]:
for f in f_sets:
    print(f)

[{frozenset({'egg', 'salt'})}, {frozenset({'egg', 'sugar'})}, {frozenset({'olive oil', 'garlic'})}, {frozenset({'sugar', 'salt'})}, {frozenset({'egg', 'unsalted butter'})}, {frozenset({'salt', 'unsalted butter'})}, {frozenset({'sugar', 'unsalted butter'})}, {frozenset({'onion', 'garlic'})}, {frozenset({'onion', 'olive oil'})}, {frozenset({'black pepper', 'salt'})}, {frozenset({'olive oil', 'salt'})}, {frozenset({'water', 'salt'})}, {frozenset({'water', 'sugar'})}, {frozenset({'all - purpose flour', 'salt'})}, {frozenset({'all - purpose flour', 'egg'})}, {frozenset({'vanilla extract', 'egg'})}]
[]
